# Net Position Analysis: Three-System Comparison

This notebook compares net position measurements from three independent systems:
1. **FFT Signal Processing** - High-precision frequency domain analysis
2. **Sonar Image Analysis** - Computer vision-based detection from multibeam sonar
3. **DVL Navigation** - Doppler velocity log-based positioning (reference)

All measurements are synchronized by timestamp and compared in:
- Distance measurements
- Pitch angle estimates
- XY position coordinates (net-relative frame)

## 1. Import Libraries and Configuration

In [3]:
from pathlib import Path
import numpy as np
from utils.net_analysis import (
    load_sonar_analysis_results,
    load_navigation_dataset,
    load_fft_dataset,
    apply_time_range_filter,
    apply_smoothing_to_all_systems,
    print_data_summaries,
    print_sample_data,
    synchronize_sonar_and_dvl,
    summarize_distance_alignment,
    prepare_three_system_comparison,
    ensure_xy_columns,
    generate_and_save_xy_plots,
    print_xy_position_statistics,
    compute_distance_pitch_statistics,
    print_distance_pitch_statistics,
    create_statistics_visualizations,
    print_quality_metrics,
    create_timeseries_visualizations,
    print_timeseries_analysis,
)

# Import config for verification
from utils.config import IMAGE_PROCESSING_CONFIG, TRACKING_CONFIG

# CONFIG VERIFICATION - Print exact values to ensure consistency
print("=== IMAGE_PROCESSING_CONFIG VALUES ===")
for key, value in IMAGE_PROCESSING_CONFIG.items():
    print(f"{key}: {value}")

print("\n=== TRACKING_CONFIG VALUES ===")
for key, value in TRACKING_CONFIG.items():
    print(f"{key}: {value}")

print("\nUtility modules ready!")

=== IMAGE_PROCESSING_CONFIG VALUES ===
binary_threshold: 128
use_advanced_momentum_merging: True
adaptive_angle_steps: 20
adaptive_base_radius: 3
adaptive_max_elongation: 1.0
momentum_boost: 10.0
adaptive_linearity_threshold: 0.75
downscale_factor: 2
top_k_bins: 8
min_coverage_percent: 0.3
gaussian_sigma: 5.0
basic_gaussian_kernel_size: 3
basic_gaussian_sigma: 1.0
basic_use_dilation: True
basic_dilation_kernel_size: 3
basic_dilation_iterations: 3
morph_close_kernel: 0
edge_dilation_iterations: 0
min_contour_area: 200
aoi_boost_factor: 10.0
max_distance_change_pixels: 20

=== TRACKING_CONFIG VALUES ===
use_elliptical_aoi: True
ellipse_expansion_factor: 0.5
aoi_boost_factor: 10.0
score_area_weight: 1.0
score_linearity_weight: 2.0
score_aspect_ratio_weight: 1.5
aoi_center_x_percent: 50
aoi_center_y_percent: 60
aoi_width_percent: 60
aoi_height_percent: 70
center_smoothing_alpha: 0.8
ellipse_size_smoothing_alpha: 0.01
ellipse_orientation_smoothing_alpha: 0.1
ellipse_max_movement_pixels: 30.

## 2. Configuration and Setup

In [11]:
# Target bag to analyze
TARGET_BAG = "2024-08-20_17-02-00"  # Change this to your desired bag ID

# Time range selection (None = use all data)
TIME_START = None    # Start time: seconds from first timestamp OR ISO datetime string
TIME_END = None     # End time: seconds from first timestamp OR ISO datetime string
# Examples:
#   TIME_START = None, TIME_END = None              # All data
#   TIME_START = 0, TIME_END = 60                   # First 60 seconds
#   TIME_START = 30, TIME_END = 90                  # 30-90 seconds from start
#   TIME_START = 120, TIME_END = None               # From 120 seconds to end
#   TIME_START = "2024-08-20T17:02:30", TIME_END = "2024-08-20T17:03:30"  # Absolute times

# Smoothing configuration (None or 1.0 = no smoothing)
SMOOTHING_ALPHA = None  # Exponential smoothing factor (0-1)
# Examples:
#   SMOOTHING_ALPHA = None   # No smoothing (raw data)
#   SMOOTHING_ALPHA = 0.1    # Heavy smoothing (90% history, 10% current)
#   SMOOTHING_ALPHA = 0.3    # Moderate smoothing (70% history, 30% current)
#   SMOOTHING_ALPHA = 0.5    # Balanced smoothing (50% history, 50% current)
#   SMOOTHING_ALPHA = 0.7    # Light smoothing (30% history, 70% current)
#   SMOOTHING_ALPHA = 0.9    # Very light smoothing (10% history, 90% current)

# Data source paths
FFT_ROOT = Path("/Volumes/LaCie/SOLAQUA/relative_fft_pose")
SONAR_ANALYSIS_DIR = Path("/Volumes/LaCie/SOLAQUA/exports/basic_full_batch")  # change as needed
PLOTS_DIR = Path("/Volumes/LaCie/SOLAQUA/exports/plots")
PLOTS_DIR.mkdir(parents=True, exist_ok=True)

import utils.net_analysis as _net_analysis  # ensure the loader sees the override
_net_analysis.SONAR_ANALYSIS_DIR = SONAR_ANALYSIS_DIR

# Raw comparison data output
COMPARISON_DATA_DIR = Path("/Volumes/LaCie/SOLAQUA/exports/basic_full_batch")

print(f"Analyzing: {TARGET_BAG}")
if TIME_START is not None or TIME_END is not None:
    if isinstance(TIME_START, (int, float)) or isinstance(TIME_END, (int, float)):
        print(f"Time range: {TIME_START or 'start'}s to {TIME_END or 'end'}s (relative)")
    else:
        print(f"Time range: {TIME_START or 'start'} to {TIME_END or 'end'} (absolute)")
else:
    print(f"Time range: All data")

if SMOOTHING_ALPHA is not None and SMOOTHING_ALPHA < 1.0:
    print(f"Smoothing: Enabled (α={SMOOTHING_ALPHA:.2f})")
else:
    print(f"Smoothing: Disabled (raw data)")

print(f"Plots: {PLOTS_DIR}")
print(f"Raw comparison data: {COMPARISON_DATA_DIR}")
print(f"Sonar analysis dir: {SONAR_ANALYSIS_DIR}")
print(f"\nNOTE: To export data for this bag, use:")
print(f"python scripts/solaqua_export.py --data-dir /Volumes/LaCie/SOLAQUA/raw_data \\")
print(f"  --exports-dir /Volumes/LaCie/SOLAQUA/exports --bag-stem {TARGET_BAG} --all")

Analyzing: 2024-08-20_17-02-00
Time range: All data
Smoothing: Disabled (raw data)
Plots: /Volumes/LaCie/SOLAQUA/exports/plots
Raw comparison data: /Volumes/LaCie/SOLAQUA/exports/basic_full_batch
Sonar analysis dir: /Volumes/LaCie/SOLAQUA/exports/basic_full_batch

NOTE: To export data for this bag, use:
python scripts/solaqua_export.py --data-dir /Volumes/LaCie/SOLAQUA/raw_data \
  --exports-dir /Volumes/LaCie/SOLAQUA/exports --bag-stem 2024-08-20_17-02-00 --all


In [12]:
# Load data from all three systems
print("=== LOADING DATA ===\n")

SUFFIX_CANDIDATES = ("_data_cones_analysis", "_analysis")

def strip_known_suffixes(bag_id: str) -> str:
    for suffix in SUFFIX_CANDIDATES:
        if bag_id.endswith(suffix):
            return bag_id[:-len(suffix)]
    return bag_id

def load_sonar_with_override(bag_id: str):
    bag_stem = strip_known_suffixes(bag_id)
    search_order = []
    for candidate in [bag_id, bag_stem] + [f"{bag_stem}{suffix}" for suffix in SUFFIX_CANDIDATES]:
        if candidate and candidate not in search_order:
            search_order.append(candidate)

    last_error = None
    for candidate in search_order:
        try:
            if SONAR_ANALYSIS_DIR is None:
                return load_sonar_analysis_results(candidate)
            return load_sonar_analysis_results(candidate, exports_dir=SONAR_ANALYSIS_DIR)
        except FileNotFoundError as exc:
            last_error = exc
    raise last_error if last_error else FileNotFoundError("Sonar analysis CSV not found")

df_sonar, sonar_meta = load_sonar_with_override(TARGET_BAG)
base_bag_for_aux = strip_known_suffixes(TARGET_BAG)
df_nav, nav_meta = load_navigation_dataset(base_bag_for_aux)
df_fft, fft_meta = load_fft_dataset(base_bag_for_aux, fft_root=FFT_ROOT)

# Apply time range selection if specified
df_sonar, df_nav, df_fft = apply_time_range_filter(df_sonar, df_nav, df_fft, TIME_START, TIME_END)

# Store raw (unsmoothed) copies for comparison data export
df_sonar_raw = df_sonar.copy() if df_sonar is not None else None
df_nav_raw = df_nav.copy() if df_nav is not None else None
df_fft_raw = df_fft.copy() if df_fft is not None else None

# Apply smoothing if specified (before any analysis)
if SMOOTHING_ALPHA is not None and SMOOTHING_ALPHA < 1.0:
    df_sonar, df_nav, df_fft = apply_smoothing_to_all_systems(df_sonar, df_nav, df_fft, SMOOTHING_ALPHA)

# Display data sources
print("Data Sources:")
for label, meta in {"Sonar": sonar_meta, "DVL": nav_meta, "FFT": fft_meta}.items():
    path = meta.get('resolved_path', 'Not found')
    rows = meta.get('rows', 0)
    status = "✓" if rows > 0 else "✗"
    print(f"  {status} {label}: {rows} records")
    if rows > 0:
        print(f"      {path}")

=== LOADING DATA ===

Data Sources:
  ✓ Sonar: 944 records
      /Volumes/LaCie/SOLAQUA/exports/basic_full_batch/2024-08-20_17-02-00_data_cones_analysis.csv
  ✓ DVL: 497 records
      /Volumes/LaCie/SOLAQUA/exports/by_bag/navigation_plane_approximation__2024-08-20_17-02-00_data.csv
  ✓ FFT: 410 records
      /Volumes/LaCie/SOLAQUA/relative_fft_pose/2024-08-20_17-02-00_relative_pose_fft.csv


In [13]:
# Summaries for each dataset
print_data_summaries(df_sonar, df_nav, df_fft)


=== DATA SUMMARIES ===

Sonar:
  rows: 944
  detections: 873
  detection_rate: 92.479
  distance_range_m: 0.000 to 1.547
  angle_range_deg: 107.707 to 191.127

DVL:
  rows: 497
  distance_range_m: 0.430 to 3.340
  distance_mean_m: 1.121
  distance_std_m: 0.305
  pitch_range_deg: -60.513 to 85.257

FFT:
  rows: 410
  distance_range_m: -1.833 to 1.596
  distance_mean_m: 0.877
  distance_std_m: 0.320
  pitch_range_deg: -82.723 to 88.077



In [14]:
# Display sample data
print_sample_data(df_sonar, df_nav, df_fft)

Sonar Analysis Sample:
                            timestamp  distance_meters  angle_degrees  \
0 2024-08-20 15:02:03.319382668+00:00         0.567219     172.237526   
1 2024-08-20 15:02:03.405480623+00:00         0.567816     172.190570   
2 2024-08-20 15:02:03.458926201+00:00         0.565049     172.167844   
3 2024-08-20 15:02:03.505110979+00:00         0.567935     172.077866   
4 2024-08-20 15:02:03.609708309+00:00         0.566654     172.106420   

   detection_success  
0               True  
1               True  
2               True  
3               True  
4               True  

DVL Navigation Sample:
                            timestamp  NetDistance  NetPitch
0 2024-08-20 15:02:05.408126593+00:00         0.72  -0.07037
1 2024-08-20 15:02:05.503421782+00:00         0.76  -0.09843
2 2024-08-20 15:02:05.603994131+00:00         0.77  -0.06585
3 2024-08-20 15:02:05.713511705+00:00         0.81  -0.07341
4 2024-08-20 15:02:05.810849667+00:00         0.82  -0.09939

FFT Data 

## 4. Two-System Synchronization (Sonar + DVL)

First, compare sonar and DVL systems independently.

In [15]:
# Synchronize sonar and DVL measurements
print("=== TWO-SYSTEM SYNCHRONIZATION (Sonar + DVL) ===\n")

sync_df = synchronize_sonar_and_dvl(df_sonar, df_nav, tolerance_seconds=1.0)
sync_summary = summarize_distance_alignment(sync_df)

print(f"Aligned pairs: {sync_summary['pairs']}")
print(f"Valid distance pairs: {sync_summary['valid_pairs']}")

if sync_summary["difference_stats"]:
    diff = sync_summary["difference_stats"]
    print(f"\nDistance Comparison (Sonar - DVL):")
    print(f"  Mean: {diff['mean']:.3f} m")
    print(f"  Std: {diff['std']:.3f} m")
    print(f"  Range: {diff['min']:.3f} to {diff['max']:.3f} m")
    print(f"  95th percentile: {diff['p95']:.3f} m")
    if 'correlation' in diff:
        print(f"  Correlation: {diff['correlation']:.3f}")
else:
    print("No overlapping distance measurements within tolerance.")

# Display sample synchronized data
if not sync_df.empty:
    print(f"\nSynchronized Data Sample:")
    display_cols = ['timestamp', 'sonar_distance_m', 'dvl_distance_m', 'sonar_angle_deg', 'dvl_pitch_deg', 'time_diff_s']
    display_cols = [c for c in display_cols if c in sync_df.columns]
    print(sync_df[display_cols].head(10))

=== TWO-SYSTEM SYNCHRONIZATION (Sonar + DVL) ===

Aligned pairs: 927
Valid distance pairs: 927

Distance Comparison (Sonar - DVL):
  Mean: -0.159 m
  Std: 0.451 m
  Range: -3.340 to 0.680 m
  95th percentile: 0.303 m
  Correlation: -0.044

Synchronized Data Sample:
                            timestamp  sonar_distance_m  dvl_distance_m  \
0 2024-08-20 15:02:04.418097496+00:00          0.568137            0.72   
1 2024-08-20 15:02:04.460487843+00:00          0.569012            0.72   
2 2024-08-20 15:02:04.520919323+00:00          0.574615            0.72   
3 2024-08-20 15:02:04.586270809+00:00          0.572133            0.72   
4 2024-08-20 15:02:04.647171021+00:00          0.571888            0.72   
5 2024-08-20 15:02:04.712663174+00:00          0.571142            0.72   
6 2024-08-20 15:02:04.801765203+00:00          0.571009            0.72   
7 2024-08-20 15:02:04.849609375+00:00          0.570511            0.72   
8 2024-08-20 15:02:04.905240774+00:00          0.568261    

/Users/eirikvarnes/code/SOLAQUA/utils/net_analysis.py:232: UserWarning:

no explicit representation of timezones available for np.datetime64



## 5. Three-System Comparison

Synchronize and compare all three systems (FFT + Sonar + DVL).

In [16]:
# Perform multi-system comparison (works with 2 or 3 systems)
print("\n=== MULTI-SYSTEM COMPARISON ===\n")

# Determine which systems are available
available_systems = []
if df_sonar is not None and not df_sonar.empty:
    available_systems.append("Sonar")
if df_nav is not None and not df_nav.empty:
    available_systems.append("DVL")
if df_fft is not None and not df_fft.empty:
    available_systems.append("FFT")

print(f"Available systems: {', '.join(available_systems)}")

if len(available_systems) < 2:
    print("✗ Need at least 2 systems for comparison")
    three_sync_df = None
    figs = {}
    visualizer = None
else:
    # Run multi-system comparison
    system_count = len(available_systems)
    print(f"Running {system_count}-system comparison...")
    
    three_sync_df, figs, visualizer = prepare_three_system_comparison(
        TARGET_BAG,
        df_sonar,
        df_nav,
        df_fft,
        tolerance_seconds=0.5
    )
    
    if three_sync_df is None or three_sync_df.empty:
        print("✗ No overlapping timestamps across available systems")
        three_sync_df = None
    else:
        print(f"✓ Synchronized {len(three_sync_df)} records across {system_count} systems\n")
        
        # Print available columns
        print("Available columns:")
        distance_cols = [c for c in three_sync_df.columns if 'distance' in c.lower()]
        pitch_cols = [c for c in three_sync_df.columns if 'pitch' in c.lower() or 'angle' in c.lower()]
        print(f"  Distance: {distance_cols}")
        print(f"  Pitch: {pitch_cols}\n")
        
        # Ensure XY columns exist
        three_sync_df = ensure_xy_columns(three_sync_df)
        
        # Generate raw comparison data (unsmoothed)
        print("\n=== GENERATING RAW COMPARISON DATA ===\n")
        print("Creating unsmoothed synchronized dataset for export...")
        
        three_sync_df_raw, _, _ = prepare_three_system_comparison(
            TARGET_BAG,
            df_sonar_raw,
            df_nav_raw,
            df_fft_raw,
            tolerance_seconds=0.5
        )
        
        if three_sync_df_raw is not None and not three_sync_df_raw.empty:
            three_sync_df_raw = ensure_xy_columns(three_sync_df_raw)
            
            # Select columns for export
            export_cols = [
                'sync_timestamp',
                # FFT data
                'fft_distance_m', 'fft_pitch_deg', 'fft_x', 'fft_y',
                # Sonar data
                'sonar_distance_m', 'sonar_pitch_deg', 'sonar_x', 'sonar_y',
                # DVL data
                'nav_distance_m', 'nav_pitch_deg', 'dvl_x', 'dvl_y'
            ]
            
            # Filter to only existing columns
            export_cols = [c for c in export_cols if c in three_sync_df_raw.columns]
            
            # Save to CSV
            output_path = COMPARISON_DATA_DIR / f"{TARGET_BAG}_raw_comparison.csv"
            three_sync_df_raw[export_cols].to_csv(output_path, index=False)
            
            print(f"✓ Saved raw comparison data: {output_path.name}")
            print(f"  Records: {len(three_sync_df_raw)}")
            print(f"  Columns: {len(export_cols)}")
            print(f"  Available systems: {', '.join([c.split('_')[0].upper() for c in export_cols if 'distance' in c])}")
        else:
            print("✗ Could not generate raw comparison data")


=== MULTI-SYSTEM COMPARISON ===

Available systems: Sonar, DVL, FFT
Running 3-system comparison...
✓ Synchronized 1851 records across 3 systems

Available columns:
  Distance: ['fft_distance_m', 'sonar_distance_m', 'nav_distance_m']
  Pitch: ['fft_pitch_deg', 'sonar_pitch_deg', 'nav_pitch_deg']

✓ XY coordinates already exist: ['fft_x_m', 'fft_y_m', 'sonar_x_m', 'sonar_y_m', 'nav_x_m', 'nav_y_m']
   Renamed columns: ['fft_x_m', 'fft_y_m', 'sonar_x_m', 'sonar_y_m', 'nav_x_m', 'nav_y_m'] → ['fft_x', 'fft_y', 'sonar_x', 'sonar_y', 'dvl_x', 'dvl_y']

=== GENERATING RAW COMPARISON DATA ===

Creating unsmoothed synchronized dataset for export...
✓ Synchronized 1851 records across 3 systems

Available columns:
  Distance: ['fft_distance_m', 'sonar_distance_m', 'nav_distance_m']
  Pitch: ['fft_pitch_deg', 'sonar_pitch_deg', 'nav_pitch_deg']

✓ XY coordinates already exist: ['fft_x_m', 'fft_y_m', 'sonar_x_m', 'sonar_y_m', 'nav_x_m', 'nav_y_m']
   Renamed columns: ['fft_x_m', 'fft_y_m', 'sonar_

## 6. Distance and Pitch Comparison Plots

In [17]:
# Display comparison plots
if three_sync_df is not None and not three_sync_df.empty and figs:
    print("=== DISPLAYING COMPARISON PLOTS ===\n")
    
    # Display all generated plots
    for name, fig in figs.items():
        if fig is not None:
            print(f"Displaying {name.replace('_', ' ').title()}...")
            try:
                fig.show()
                
                # Save plots
                save_path = PLOTS_DIR / f"{TARGET_BAG}_{name}.html"
                fig.write_html(str(save_path))
                print(f"  ✓ Saved: {save_path.name}\n")
            except Exception as e:
                print(f"  ✗ Warning: Could not display/save - {e}\n")
    
    print("✓ All plots displayed and saved")
else:
    print("✗ No synchronized data or plots available")
    print("  This usually means:")
    print("  - Timestamps don't overlap between systems")
    print("  - Data quality issues")
    print("  - Missing required columns in datasets")

=== DISPLAYING COMPARISON PLOTS ===

Displaying Distance Comparison...


  ✓ Saved: 2024-08-20_17-02-00_distance_comparison.html

Displaying Pitch Comparison...


  ✓ Saved: 2024-08-20_17-02-00_pitch_comparison.html

✓ All plots displayed and saved


## 7. Net-Relative XY Position Plots

Visualize the robot's trajectory relative to the net plane in XY coordinates from all systems.

In [18]:
# Display XY position plots
if three_sync_df is not None and not three_sync_df.empty and visualizer is not None:
    # Configure lateral speed (robot moving along net)
    LATERAL_SPEED_M_S = 0.2  # meters per second (adjust based on actual speed)
    
    # Generate and save all XY plots
    xy_plot_results = generate_and_save_xy_plots(
        three_sync_df, 
        visualizer, 
        TARGET_BAG, 
        PLOTS_DIR, 
        lateral_speed_m_s=LATERAL_SPEED_M_S
    )
else:
    print("✗ No synchronized data or visualizer available for XY plots")

=== DISPLAYING XY POSITION PLOTS ===

Systems with XY coordinates: Sonar, DVL, FFT

Generating 3D XY trajectory plot...
  Lateral speed: 0.2 m/s


✓ Saved: 2024-08-20_17-02-00_xy_trajectories_3d.html


Generating XY component comparison plots...

--- Perpendicular Distance to Net Over Time ---


✓ X position comparison saved: 2024-08-20_17-02-00_x_position_comparison.html

--- Lateral Position Along Net Over Time ---


✓ Y position comparison saved: 2024-08-20_17-02-00_y_position_comparison.html

✓ XY position plots complete


## 8. Position Statistics and Analysis

In [19]:
# Display detailed position statistics
if three_sync_df is not None and not three_sync_df.empty:
    # XY Position Summary
    print_xy_position_statistics(three_sync_df)
    
    # Distance and Pitch Statistics
    distance_pitch_stats = compute_distance_pitch_statistics(three_sync_df)
    print_distance_pitch_statistics(distance_pitch_stats)
    
    # Quality metrics (no ground truth needed)
    print_quality_metrics(three_sync_df)
    
    # Time series analysis
    print_timeseries_analysis(three_sync_df)
    
    # Generate and display statistical visualizations
    print("\n=== GENERATING STATISTICAL VISUALIZATIONS ===\n")
    stat_figs = create_statistics_visualizations(three_sync_df, TARGET_BAG)
    
    for name, fig in stat_figs.items():
        if fig is not None:
            print(f"Displaying {name.replace('_', ' ').title()}...")
            try:
                display(fig)
                save_path = PLOTS_DIR / f"{TARGET_BAG}_stats_{name}.html"
                fig.write_html(str(save_path))
                print(f"  ✓ Saved: {save_path.name}\n")
            except Exception as e:
                print(f"  ✗ Warning: Could not display/save - {e}\n")
    
    # Generate and display time series visualizations
    print("=== GENERATING TIME SERIES VISUALIZATIONS ===\n")
    ts_figs = create_timeseries_visualizations(three_sync_df, TARGET_BAG)
    
    for name, fig in ts_figs.items():
        if fig is not None:
            print(f"Displaying {name.replace('_', ' ').title()}...")
            try:
                display(fig)
                save_path = PLOTS_DIR / f"{TARGET_BAG}_timeseries_{name}.html"
                fig.write_html(str(save_path))
                print(f"  ✓ Saved: {save_path.name}\n")
            except Exception as e:
                print(f"  ✗ Warning: Could not display/save - {e}\n")
    
    # Additional detailed analysis if visualizer available
    if visualizer is not None:
        try:
            print("\n=== DETAILED POSITION ANALYSIS ===\n")
            visualizer.print_position_summary(three_sync_df)
        except Exception as e:
            print(f"Could not print detailed analysis: {e}")
else:
    print("✗ No synchronized data available for position analysis")

=== POSITION STATISTICS ===

FFT:
  Data points: 1769
  X position: 0.820 ± 0.379 m
  Y position: 0.039 ± 0.122 m
  Distance from origin: 0.849 ± 0.335 m

Sonar:
  Data points: 1851
  X position: 0.964 ± 0.229 m
  Y position: -0.096 ± 0.096 m
  Distance from origin: 0.974 ± 0.223 m

DVL:
  Data points: 1813
  X position: 1.097 ± 0.293 m
  Y position: -0.080 ± 0.285 m
  Distance from origin: 1.126 ± 0.332 m

=== DISTANCE & PITCH STATISTICS ===

Distance Measurements:
  FFT:
    Mean: 0.828 m
    Std: 0.384 m
    Min: -1.833 m
    Max: 1.596 m
    Range: 3.429 m
  Sonar:
    Mean: 0.974 m
    Std: 0.223 m
    Min: 0.000 m
    Max: 1.547 m
    Range: 1.547 m
  DVL:
    Mean: 1.126 m
    Std: 0.332 m
    Min: 0.430 m
    Max: 3.340 m
    Range: 2.910 m

Pitch/Angle Measurements:
  FFT:
    Mean: 3.3°
    Std: 16.5°
    Min: -82.7°
    Max: 88.1°
    Range: 170.8°
  Sonar:
    Mean: -6.8°
    Std: 10.4°
    Min: -72.3°
    Max: 11.1°
    Range: 83.4°
  DVL:
    Mean: -2.9°
    Std: 11.9°
  

  ✓ Saved: 2024-08-20_17-02-00_stats_distributions.html

Displaying Stability...


  ✓ Saved: 2024-08-20_17-02-00_stats_stability.html

Displaying Scatter...


  ✓ Saved: 2024-08-20_17-02-00_stats_scatter.html

=== GENERATING TIME SERIES VISUALIZATIONS ===

Displaying Autocorrelation...


  ✓ Saved: 2024-08-20_17-02-00_timeseries_autocorrelation.html

Displaying Rolling Stats...


  ✓ Saved: 2024-08-20_17-02-00_timeseries_rolling_stats.html


=== DETAILED POSITION ANALYSIS ===

Position data summary:


## 9. Summary

In [20]:
print("\n" + "="*70)
print("ANALYSIS SUMMARY")
print("="*70)

print(f"\nTarget Bag: {TARGET_BAG}")

print(f"\nData Availability:")
for label, meta in {"Sonar": sonar_meta, "DVL": nav_meta, "FFT": fft_meta}.items():
    rows = meta.get('rows', 0)
    status = "✓" if rows > 0 else "✗"
    print(f"  {status} {label}: {rows} records")

print(f"\nSynchronization Results:")
print(f"  Two-system (Sonar+DVL): {len(sync_df)} records")
if three_sync_df is not None:
    print(f"  Multi-system: {len(three_sync_df)} records")
else:
    print(f"  Multi-system: N/A")

if three_sync_df is not None and not three_sync_df.empty:
    # Count available plots
    plot_count = len(figs) if figs else 0
    
    # Check for XY plots
    has_xy = ('sonar_x' in three_sync_df.columns or 
              'dvl_x' in three_sync_df.columns or 
              'fft_x' in three_sync_df.columns)
    
    if has_xy:
        plot_count += 3  # XY trajectory + X comparison + Y Comparison
    
    print(f"\nGenerated Plots: {plot_count}")
    if figs:
        for name in figs.keys():
            print(f"  - {name.replace('_', ' ').title()}")
    if has_xy:
        print(f"  - XY Trajectories")
        print(f"  - X Position Comparison")
        print(f"  - Y Position Comparison")
    
    print(f"\nSaved to: {PLOTS_DIR}")

print("\n✓ Notebook complete")


ANALYSIS SUMMARY

Target Bag: 2024-08-20_17-02-00

Data Availability:
  ✓ Sonar: 944 records
  ✓ DVL: 497 records
  ✓ FFT: 410 records

Synchronization Results:
  Two-system (Sonar+DVL): 927 records
  Multi-system: 1851 records

Generated Plots: 5
  - Distance Comparison
  - Pitch Comparison
  - XY Trajectories
  - X Position Comparison
  - Y Position Comparison

Saved to: /Volumes/LaCie/SOLAQUA/exports/plots

✓ Notebook complete


In [22]:
# Batch full-comparison export for every sonar analysis file
print("\n=== BATCH FULL COMPARISON EXPORT ===\n")

analysis_files = sorted(SONAR_ANALYSIS_DIR.glob('*_data_cones_analysis.csv'))
if not analysis_files:
    analysis_files = sorted(SONAR_ANALYSIS_DIR.glob('*_analysis.csv'))
if not analysis_files:
    raise FileNotFoundError(f"No sonar analysis CSVs found in {SONAR_ANALYSIS_DIR}")

def extract_bag_id(path: Path) -> str:
    stem = path.stem
    for suffix in ('_data_cones_analysis', '_analysis'):
        if stem.endswith(suffix):
            return stem[:-len(suffix)]
    return stem

def build_full_comparison_path(path: Path) -> Path:
    stem = path.stem
    for suffix in ('_data_cones_analysis', '_analysis'):
        if stem.endswith(suffix):
            stem = stem[:-len(suffix)]
            break
    return path.with_name(f"{stem}_raw_comparison.csv")

written = 0
for analysis_csv in analysis_files:
    if analysis_csv.name.startswith('._'):
        print(f"↷ skipping resource file {analysis_csv.name}")
        continue
    bag_id = extract_bag_id(analysis_csv)
    print(f"→ {bag_id}")
    try:
        sonar_df, _ = load_sonar_analysis_results(bag_id)
        nav_df, _ = load_navigation_dataset(bag_id)
        fft_df, _ = load_fft_dataset(bag_id, fft_root=FFT_ROOT)
    except FileNotFoundError as exc:
        print(f"  ✗ missing data: {exc}")
        continue

    sonar_df, nav_df, fft_df = apply_time_range_filter(sonar_df, nav_df, fft_df, TIME_START, TIME_END)
    sonar_raw = sonar_df.copy() if sonar_df is not None else None
    nav_raw = nav_df.copy() if nav_df is not None else None
    fft_raw = fft_df.copy() if fft_df is not None else None

    if SMOOTHING_ALPHA is not None and SMOOTHING_ALPHA < 1.0:
        sonar_df, nav_df, fft_df = apply_smoothing_to_all_systems(sonar_df, nav_df, fft_df, SMOOTHING_ALPHA)

    three_sync_df_raw, _, _ = prepare_three_system_comparison(
        bag_id, sonar_raw, nav_raw, fft_raw, tolerance_seconds=0.5
    )
    if three_sync_df_raw is None or three_sync_df_raw.empty:
        print("  ✗ no overlapping samples to export")
        continue

    three_sync_df_raw = ensure_xy_columns(three_sync_df_raw)
    export_cols = [
        'sync_timestamp',
        'fft_distance_m', 'fft_pitch_deg', 'fft_x', 'fft_y',
        'sonar_distance_m', 'sonar_pitch_deg', 'sonar_x', 'sonar_y',
        'nav_distance_m', 'nav_pitch_deg', 'dvl_x', 'dvl_y'
    ]
    export_cols = [c for c in export_cols if c in three_sync_df_raw.columns]
    output_csv = build_full_comparison_path(analysis_csv)
    three_sync_df_raw[export_cols].to_csv(output_csv, index=False)
    written += 1
    print(f"  ✓ saved {output_csv.name}")

print(f"\nBatch complete: {written} file(s) written under {SONAR_ANALYSIS_DIR}")


=== BATCH FULL COMPARISON EXPORT ===

↷ skipping resource file ._2024-08-20_13-39-34_data_cones_analysis.csv
↷ skipping resource file ._2024-08-20_13-40-35_data_cones_analysis.csv
↷ skipping resource file ._2024-08-20_13-42-51_data_cones_analysis.csv
↷ skipping resource file ._2024-08-20_13-55-34_data_cones_analysis.csv
↷ skipping resource file ._2024-08-20_13-57-42_data_cones_analysis.csv
↷ skipping resource file ._2024-08-20_14-16-05_data_cones_analysis.csv
↷ skipping resource file ._2024-08-20_14-22-12_data_cones_analysis.csv
↷ skipping resource file ._2024-08-20_14-24-35_data_cones_analysis.csv
↷ skipping resource file ._2024-08-20_14-31-29_data_cones_analysis.csv
↷ skipping resource file ._2024-08-20_14-57-38_data_cones_analysis.csv
↷ skipping resource file ._2024-08-20_15-00-24_data_cones_analysis.csv
↷ skipping resource file ._2024-08-20_15-14-40_data_cones_analysis.csv
↷ skipping resource file ._2024-08-20_16-57-46_data_cones_analysis.csv
↷ skipping resource file ._2024-08-20_